In [5]:
!markitdown ONEREPORTBLANDT.PDF > onereport.md

In [6]:
import getpass
from langchain_community.embeddings import DeepInfraEmbeddings

CUSTOM_OPENAI_KEY = getpass.getpass("Enter API key for OpenAI: ")

deep_infra_embedding = DeepInfraEmbeddings(
    model_id="intfloat/multilingual-e5-large", deepinfra_api_token=CUSTOM_OPENAI_KEY
)

In [7]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(deep_infra_embedding)

In [9]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader(
    "./onereport.md",
    mode="single",
    strategy="fast",
)

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-72B-Instruct",
    base_url="https://api.deepinfra.com/v1/openai",
    api_key=CUSTOM_OPENAI_KEY,
)

In [15]:
from langchain import hub
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/home/pii/Develop/talks/RAG/.venv/lib/python3.12/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [16]:
response = graph.invoke({"question": "จาก One Report ของบริษัทนี้ มีการพูดถึงรางวัลด้าน ESG ใดบ้าง เช่น DJSI, SET ESG Rating และอื่น ๆ"})
print(response["answer"])

จากข้อมูลที่ให้มา ไม่มีการระบุถึงรางวัลด้าน ESG ใดๆ เช่น DJSI, SET ESG Rating หรือรางวัลอื่นๆ ใน One Report ของบริษัท.
